In [ ]:
!pip install pyradox-tabular -q
import pandas as pd
import numpy as np
import sklearn
from pyradox_tabular.data import DataLoader
from pyradox_tabular.data_config import DataConfig
from pyradox_tabular.model_config import WideAndDeepNetworkConfig
from pyradox_tabular.nn import WideAndDeepTabularNetwork

In [ ]:
test= pd.read_csv("../input/tabular-playground-series-feb-2022/test.csv")           
sample_sub=pd.read_csv("../input/tabular-playground-series-feb-2022/sample_submission.csv")
train = pd.read_csv("../input/tabular-playground-series-feb-2022/train.csv")

In [ ]:
from sklearn.utils import shuffle
train = shuffle(train)

In [ ]:
targets = train.target.unique()
target_dict = {t: i for i, t in enumerate(targets)}
target_dict_inv = {v: k for k, v in target_dict.items()}
train["target_map"] = train.target.map(target_dict)
features = [c for c in train.columns if c not in ("target","row_id","target_map")]
target="target"
x_test=test[features]
y = train[target]
X = train[features]
TARGET_LABELS = sorted(list(train["target"].unique()))
y = pd.get_dummies(train.target)
numerical_features = X.columns

#Jsut for testing
X_train = X[:100000]
y_train =y[:100000]
X_valid = X[100000:]
y_valid =y[100000:]

In [ ]:
data_config = DataConfig(numeric_feature_names=list(numerical_features), 
                         categorical_features_with_vocabulary={}
                        )

In [ ]:
data_train = DataLoader.from_df(X_train, y_train, batch_size=1024)
data_valid = DataLoader.from_df(X_valid, y_valid, batch_size=1024)
data_test = DataLoader.from_df(x_test, batch_size=1024)

In [ ]:
model_config = WideAndDeepNetworkConfig(num_outputs=10, out_activation='softmax', hidden_units=[64, 64,18])
model = WideAndDeepTabularNetwork.from_config(data_config, model_config, name="wide_deep_network")
model.compile(optimizer="adam", loss="categorical_crossentropy")
model.fit(data_train, validation_data=data_valid)
preds = model.predict(data_test)

In [ ]:
preds